In [12]:
overview="""{# D, table, newcases, np, pd  #}
{% set COL_REGION = COL_REGION or 'Country/Region' %}
{% set KPI_CASE = KPI_CASE or 'World' %}
{% set KPIS_INFO = KPIS_INFO or [{'title': 'China', 'prefix': 'China'}, {'title': 'Europe', 'prefix': 'EU'}, {'title': "Cote d'Ivoire", 'prefix': 'CIV'}] %}
{% set LEGEND_DOMAIN = LEGEND_DOMAIN or [10, 100, 1000, np.inf] %}
{% set WIDTH_REGION, WIDTH_STRIP = 120, 140 %}
{% set STRIP_WIDTH = (WIDTH_REGION // newcases.shape[1] + 1) %}
{% set LEGEND_RANGE = ['rgba(255, 152, 0, 0.1)', 'rgba(255, 152, 0, 0.4)', 'rgba(255, 152, 0, 0.7)', 'rgba(255, 152, 0, 1)'] %}
{% set TOPLINKS = TOPLINKS or [
  {'title': 'World', 'href': '../covid-overview/'}, {'title': "Cote d'Ivoire", 'href': '../covid-overview-civ/'},
  {'title': 'Europe', 'href': '../covid-overview-europe/'}] %}
{% set lastdays = (D['updated'] - D['since']).days %}

{% macro kpi(name, number, growth, growcls='') -%}
  <div class="kpi">
    <div class="kname">{{ name }}</div>
    <div class="num">{{ '{0:,.0f}'.format(number) }}</div>
    <div class="grow {{ growcls }}">(<b>{{ '{0:+,.0f}'.format(growth) }}</b>)</div>
  </div>
{%- endmacro %}

{% macro kpiblocksm(prefix='', title=KPI_CASE) -%}
  <div class="kpi-sm">
    <div class="kpi-hed">{{ title }}</div>
    <div class="d-flex kpi-box">
      {{ kpi(name='Cases', number=D[prefix + ' Cases'], growth=D[prefix + ' Cases (+)']) }}
      {{ kpi(name='Deaths', number=D[prefix + ' Deaths'], growth=D[prefix + ' Deaths (+)']) }}
    </div>
  </div>
{%- endmacro %}

{% macro toplinks() -%}
<div class="text-center toplinksgithub">
  {% for link in TOPLINKS %}<a href="{{ link['href'] }}">{{ link['title'] }}</a>{% endfor %}
</div>
{%- endmacro %}

{% macro narrative() -%}
{% if KPI_CASE == 'World' %}
  Pendant les <b>{{ lastdays }} deniers jours</b>, <b class="color-neg">{{ '{0:,.0f}'.format(D['Cases (+)']) }}</b> nouveaux cas de coronavirus ont été signalés dans le monde.
  Dont <b class="color-neg">{{ '{0:,.0f}'.format(D['EU Cases (+)']) }}</b> ({{ "{0:.0%}".format(D['EU Cases (+)'] / D['Cases (+)']) }}) sont originaires d'Europe <b>Europe</b>.
  <b> La Chine</b> a signalé <b class="color-neg">{{ '{0:,.0f}'.format(D['China Cases (+)']) }}</b> nouveaux cas les {{ lastdays }} derniers jours.
{% elif KPI_CASE == "Cote d'Ivoire" %}
  Pendant les <b>{{ lastdays }} deniers jours</b>, <b class="color-neg">{{ '{0:,.0f}'.format(D['Cases (+)']) }}</b> nouveaux cas de coronavirus ont été signalés en Cote d'Ivoire.
  Dont <b class="color-neg">{{ '{0:,.0f}'.format(D['NY Cases (+)']) }}</b> ({{ "{0:.0%}".format(D['NY Cases (+)'] / D['Cases (+)']) }}) are from <b>New York</b> State.
  <b>Washington</b> has reported <b class="color-neg">{{ '{0:,.0f}'.format(D['WA Cases (+)']) }}</b> new cases in the last {{ lastdays }} days.
{% elif KPI_CASE == 'Europe' %}
  In the last <b>{{ lastdays }} days</b>, <b class="color-neg">{{ '{0:,.0f}'.format(D['Cases (+)']) }}</b> new Coronavirus cases have been reported in the Europe.
  Of which <b class="color-neg">{{ '{0:,.0f}'.format(D['IT Cases (+)']) }}</b> ({{ "{0:.0%}".format(D['IT Cases (+)'] / D['Cases (+)']) }}) are from <b>Italy</b>.
  <b>Spain</b> has reported <b class="color-neg">{{ '{0:,.0f}'.format(D['SP Cases (+)']) }}</b> new cases in the last {{ lastdays }} days.
{% elif KPI_CASE == 'Asia' %}
  In the last <b>{{ lastdays }} days</b>, <b class="color-neg">{{ '{0:,.0f}'.format(D['Cases (+)']) }}</b> new Coronavirus cases have been reported in Asia.
  Of which <b class="color-neg">{{ '{0:,.0f}'.format(D['CN Cases (+)']) }}</b> ({{ "{0:.0%}".format(D['CN Cases (+)'] / D['Cases (+)']) }}) are from <b>China</b>.
  <b>India</b> has reported <b class="color-neg">{{ '{0:,.0f}'.format(D['IN Cases (+)']) }}</b> new cases in the last {{ lastdays }} days.
{% else %}
  ''
{% endif %}
{%- endmacro %}

{% macro plotstrip(arr) -%}
  <div class="d-flex" style="height:15px;">
    {% set colors = np.digitize(arr, LEGEND_DOMAIN) %}
    {% for i, v in enumerate(arr) %}
    <div style="width:{{ STRIP_WIDTH }}px;background:{{ LEGEND_RANGE[colors[i]] if (v) else '#eee' }};border-right:1px solid rgba(255,255,255,0.5);"></div>
    {% endfor %}
  </div>
{%- endmacro %}

{% macro legend() -%}
<svg width="100" height="20" viewBox="0,0,100,20" style="overflow: visible; display: block;">
  <g>
    {% for i, _ in enumerate(LEGEND_DOMAIN) %}
    <rect x="{{ 25 * i }}"  y="8" width="25" height="10" fill="{{ LEGEND_RANGE[i] }}"></rect>
    {% endfor %}
  </g>
  <g style="font-size:10px;text-anchor:middle;">
    {% for i, x in enumerate(LEGEND_DOMAIN[:-1], 1) %}
    <g transform="translate({{ 25 * i }}, 6)"><text>{{ x }}</text></g>
    {% endfor %}
    </g>
</svg>
{%- endmacro %}

<div class="overview">
  {{ toplinks() }}
  <div>
    <div class="kpi-hed text-center">{{ KPI_CASE }}</div>
    <div class="d-flex kpi-box">
      {{ kpi(name='Confirmed Cases', number=D['Cases'], growth=D['Cases (+)']) }}
      {{ kpi(name='Deaths', number=D['Deaths'], growth=D['Deaths (+)']) }}
    </div>
  </div>
  <p class="text-center text-uppercase fs9">Updated on <b>{{ D['updated'].strftime('%B %d, %Y') }}</b> ( +change since {{ lastdays }} days ago.)</p>
  <div class="d-flex" style="justify-content:space-between;">
    {% for kpi in KPIS_INFO %}
    {{ kpiblocksm(**kpi) }}
    {% endfor %}
  </div>
  <p class="text-center" style="font-size: 14px;max-width: 400px;">{{ narrative() }}</p>
  <table class="table" style="width:575px;">
    <thead>
      <tr>
        <th class="text-right" style="width:{{ WIDTH_REGION }}px;"></th>
        <th class="text-left" style="width:{{ WIDTH_STRIP }}px;">{{ legend() }}</th>
        <th colspan="7"></th>
      </tr>
      <tr>
        <th class="text-right" style="width:{{ WIDTH_REGION }}px;">{{ {'Country/Region': 'Country', 'Province/State': 'State'}.get(COL_REGION, 'Location') }}</th>
        <th class="text-left" style="width:{{ WIDTH_STRIP }}px;">New Cases</th>
        <th class="text-left" colspan="2">Total Cases</th>
        <th colspan="2">Deaths</th>
        <th class="fs9" >Fatality</th>
      </tr>
    </thead>
    <tbody>
      <tr style="font-size:9px;">
        <td></td>
        <td style="display:flex;justify-content:space-between;">
          <div>{{ pd.to_datetime(newcases.columns[0]).strftime('%b. %d') }}</div>
          <div>{{ pd.to_datetime(newcases.columns[-1]).strftime('%b. %d') }}</div>
        </td>
        <td></td>
        <td colspan="4" class="text-left change" style="font-size: 9px;">(+NEW) since {{ D['since'].strftime('%b, %d') }}</td>
        <td></td>
        <td></td>
      </tr>
    {% for index, row in table.iterrows() %}
      <tr>
        <td class="mw"><b>{{ row[COL_REGION] }}</b></td>
        <td style="vertical-align: middle;">{{ plotstrip(arr=newcases.loc[row[COL_REGION]].values) }}</td>
        <td class="pl1"><b>{{ '{0:,.0f}'.format(row['Cases']) }}</b></td>
        <td class="change neg">(<b>{{ '{0:+,.0f}'.format(row['Cases (+)']) }}</b>)</td>
        <td class="pl1">{{ '{0:,.0f}'.format(row['Deaths']) }}</td>
        <td class="change neg">(<b>{{ '{0:+,.0f}'.format(row['Deaths (+)']) }}</b>)</td>
        <td class="pl1">{{ row['Fatality Rate'] }}%</td>
      </tr>
    {% endfor %}
    </tbody>
  </table>
</div>
<style>
.overview {
  min-width: 500px;
  font-size: 10px;
  font-family: "Segoe UI", SegoeUI, Roboto, "Segoe WP", "Helvetica Neue", "Helvetica", "Tahoma", "Arial", sans-serif !important;
}
.overview .toplinksgithub a {
  background: #d3d3d3;
  font-size: 14px;
  color: #1d87ae;
  margin: 10px;
  padding: 2px 10px;
}
.overview p {
  margin: 6px auto !important;
  padding: 0;
}
@media screen and (max-width: 660px) {
  .overview p { max-width: none !important; }
}
.overview b {
  font-weight: bolder;
}
.overview .kpi-hed {
  font-weight: bold;
  font-size: 20px;
}
.overview .kpi-box {
  justify-content: space-around;
  background: #ececec;
  padding: 10px 0 !important;
  margin: 5px 0 !important;
  min-width: 180px;
}
.overview .kpi .num {
  font-size: 40px;
  line-height: 40px;
  font-weight: bold;
}
.overview .kpi .grow {
  line-height: 12px;
  font-size: 12px;
}
.overview .table .change.pos , .overview .kpi .grow.pos {
  color: #118822;
}
.overview .table .change.neg, .overview .kpi .grow, .color-neg {
  color: #cc1100;
}
.overview p .color-neg {
  background: #ececec;
  padding: 0 5px;
}
.overview .kpi .kname {
  font-size: 12px;
}
.overview .kpi-sm .kpi-hed {
  font-size: 14px;
  line-height: 10px;
  padding-top: 10px !important;
}
.overview .kpi-sm .num {
  font-size: 20px;
  line-height: 20px;
}
.overview .kpi-sm .kname {
  font-size: 11px;
  line-height: 10px;
}
.overview .table {
  border-collapse: collapse;
  margin: auto !important;
  text-align: right;
  margin-top: 14px;
  color: black;
  font-size: 13px;
  display: table !important;
}
.overview .table .change {
  color: #999;
  font-size: 80%;
  text-align: start;
  vertical-align: inherit;
  font-weight: normal;
  padding-left: 1px !important;
}
.overview .table th {
  font-weight: normal;
}
.overview .table tbody tr {
  border-bottom: 1px solid #eee;
  background: none;
}
.overview .table td, .overview .table th {
  padding: 1px 1px 1px 10px !important;
  vertical-align: middle;
  border: none;
  background: none;
}
.overview .table th {
  text-align: center;
  text-transform: uppercase;
}
.overview .table thead {
  border-bottom: 1px solid black;
}
.overview .fs9 {
  font-size: 9px;
}
.overview .d-flex {
  display: flex;
}
.overview .text-center { text-align: center !important; }
.overview .text-left { text-align: left !important; }
.overview .text-right { text-align: right !important; }
.overview .text-uppercase { text-transform: uppercase !important; }
.overview div {
  margin: 0 !important;
  padding: 0 !important;
}
</style> """

# COVID-19 Overview
> Tracking coronavirus total cases, deaths and new cases by country. Additionally, a detailed view is provided for the US(by state), Europe

- comments: true
- author: Pratap Vardhan
- categories: [overview, interactive]
- image: images/covid-overview.png
- permalink: /covid-overview/
- hide: false

# hide
print('''
Example of using jupyter notebook, pandas (data transformations), jinja2 (html, visual)
to create visual dashboards with fastpages
You see also the live version on https://gramener.com/enumter/covid19/
''')

In [13]:
#hide
import numpy as np
import pandas as pd
from jinja2 import Template
from IPython.display import HTML

In [14]:
#hide

# FETCH
base_url = 'https://raw.githubusercontent.com/pratapvardhan/notebooks/master/covid19/'
paths = {
    'mapping': base_url + 'mapping_countries.csv',
    'overview': base_url + overview
}

def get_mappings(url):
    df = pd.read_csv(url)
    return {
        'df': df,
        'replace.country': dict(df.dropna(subset=['Name']).set_index('Country')['Name']),
        'map.continent': dict(df.set_index('Name')['Continent'])
    }

mapping = get_mappings(paths['mapping'])

def get_template(path):
    from urllib.parse import urlparse
    if bool(urlparse(path).netloc):
        from urllib.request import urlopen
        return urlopen(path).read().decode('utf8')
    return open(path).read()

def get_frame(name):
    url = (
        'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/'
        f'csse_covid_19_time_series/time_series_covid19_{name}_global.csv')
    df = pd.read_csv(url)
    # rename countries
    df['Country/Region'] = df['Country/Region'].replace(mapping['replace.country'])
    return df

def get_dates(df):
    dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]
    LAST_DATE_I = -1
    # sometimes last column may be empty, then go backwards
    for i in range(-1, -len(dt_cols), -1):
        if not df[dt_cols[i]].fillna(0).eq(0).all():
            LAST_DATE_I = i
            break
    return LAST_DATE_I, dt_cols

In [15]:
#hide
COL_REGION = 'Country/Region'
# Confirmed, Recovered, Deaths
df = get_frame('confirmed')
# dft_: timeseries, dfc_: today country agg
dft_cases = df
dft_deaths = get_frame('deaths')
dft_recovered = get_frame('recovered')
LAST_DATE_I, dt_cols = get_dates(df)

dt_today = dt_cols[LAST_DATE_I]
dt_5ago = dt_cols[LAST_DATE_I-5]


dfc_cases = dft_cases.groupby(COL_REGION)[dt_today].sum()
dfc_deaths = dft_deaths.groupby(COL_REGION)[dt_today].sum()
dfp_cases = dft_cases.groupby(COL_REGION)[dt_5ago].sum()
dfp_deaths = dft_deaths.groupby(COL_REGION)[dt_5ago].sum()

In [16]:
#hide
df_table = (pd.DataFrame(dict(Cases=dfc_cases, Deaths=dfc_deaths, PCases=dfp_cases, PDeaths=dfp_deaths))
             .sort_values(by=['Cases', 'Deaths'], ascending=[False, False])
             .reset_index())
df_table.rename(columns={'index': 'Country/Region'}, inplace=True)
for c in 'Cases, Deaths'.split(', '):
    df_table[f'{c} (+)'] = (df_table[c] - df_table[f'P{c}']).clip(0)  # DATA BUG
df_table['Fatality Rate'] = (100 * df_table['Deaths'] / df_table['Cases']).round(1)
df_table['Continent'] = df_table['Country/Region'].map(mapping['map.continent'])
df_table.head(15)

Country/Region   Cases  Deaths  PCases  PDeaths  Cases (+)  Deaths (+)  \
0               US  366614   10783  213372     4757     153242        6026   
1            Spain  136675   13341  104118     9387      32557        3954   
2            Italy  132547   16523  110574    13155      21973        3368   
3          Germany  103374    1810   77872      920      25502         890   
4           France   98963    8926   57749     4043      41214        4883   
5            China   82665    3335   82361     3316        304          19   
6             Iran   60500    3739   47593     3036      12907         703   
7   United Kingdom   52279    5385   29865     2357      22414        3028   
8           Turkey   30217     649   15679      277      14538         372   
9      Switzerland   21657     765   17768      488       3889         277   
10         Belgium   20814    1632   13964      828       6850         804   
11     Netherlands   18926    1874   13696     1175       5230         699   
12          Canada   16563     339    9560      109       7003         230   
13         Austria   12297     220   10711      146       1586          74   
14          Brazil   12161     564    6836      240       5325         324   

    Fatality Rate      Continent  
0             2.9  North America  
1             9.8         Europe  
2            12.5         Europe  
3             1.8         Europe  
4             9.0         Europe  
5             4.0           Asia  
6             6.2           Asia  
7            10.3         Europe  
8             2.1           Asia  
9             3.5         Europe  
10            7.8         Europe  
11            9.9         Europe  
12            2.0  North America  
13            1.8         Europe  
14            4.6  South America

In [17]:
df_table['Continent'].unique()

array(['North America', 'Europe', 'Asia', 'South America', 'Oceania',
       'Africa', nan], dtype=object)

In [18]:
df_table['Country/Region'].unique()

array(['US', 'Spain', 'Italy', 'Germany', 'France', 'China', 'Iran',
       'United Kingdom', 'Turkey', 'Switzerland', 'Belgium',
       'Netherlands', 'Canada', 'Austria', 'Brazil', 'Portugal',
       'South Korea', 'Israel', 'Sweden', 'Russia', 'Norway', 'Australia',
       'Ireland', 'Denmark', 'Czechia', 'Chile', 'India', 'Poland',
       'Romania', 'Malaysia', 'Pakistan', 'Ecuador', 'Philippines',
       'Japan', 'Luxembourg', 'Saudi Arabia', 'Peru', 'Indonesia',
       'Thailand', 'Serbia', 'Finland', 'Mexico', 'United Arab Emirates',
       'Panama', 'Qatar', 'Dominican Republic', 'Greece', 'South Africa',
       'Colombia', 'Iceland', 'Argentina', 'Algeria', 'Singapore',
       'Egypt', 'Ukraine', 'Croatia', 'Morocco', 'Estonia', 'New Zealand',
       'Iraq', 'Slovenia', 'Moldova', 'Lithuania', 'Armenia', 'Bahrain',
       'Hungary', 'Diamond Princess', 'Belarus', 'Bosnia and Herzegovina',
       'Kuwait', 'Kazakhstan', 'Cameroon', 'Azerbaijan', 'Tunisia',
       'North Macedon

In [19]:
#hide
#delete problematic countries from table
df_table = df_table[~df_table['Country/Region'].isin(['Cape Verde', 'Cruise Ship', 'Kosovo'])]

In [20]:
#hide
# world, china, europe, us
metrics = ['Cases', 'Deaths', 'Cases (+)', 'Deaths (+)']

s_africa= df_table[df_table['Continent'].eq('Africa')][metrics].sum().add_prefix('AFR ')
s_civ= df_table[df_table['Country/Region'].eq("Cote d'Ivoire")][metrics].sum().add_prefix('CIV ')
s_china = df_table[df_table['Country/Region'].eq('China')][metrics].sum().add_prefix('China ')
s_us = df_table[df_table['Country/Region'].eq('US')][metrics].sum().add_prefix('US ')
s_eu = df_table[df_table['Continent'].eq('Europe')][metrics].sum().add_prefix('EU ')
summary = {'updated': pd.to_datetime(dt_today), 'since': pd.to_datetime(dt_5ago)}
summary = {**summary, **df_table[metrics].sum(), **s_china, **s_us, **s_eu, **s_africa, **s_civ}
summary

{'updated': Timestamp('2020-04-06 00:00:00'),
 'since': Timestamp('2020-04-01 00:00:00'),
 'Cases': 1344903,
 'Deaths': 74564,
 'Cases (+)': 412423,
 'Deaths (+)': 27756,
 'China Cases': 82665,
 'China Deaths': 3335,
 'China Cases (+)': 304,
 'China Deaths (+)': 19,
 'US Cases': 366614,
 'US Deaths': 10783,
 'US Cases (+)': 153242,
 'US Deaths (+)': 6026,
 'EU Cases': 675104,
 'EU Deaths': 52661,
 'EU Cases (+)': 186878,
 'EU Deaths (+)': 19007,
 'AFR Cases': 9043,
 'AFR Deaths': 457,
 'AFR Cases (+)': 3184,
 'AFR Deaths (+)': 243,
 'CIV Cases': 323,
 'CIV Deaths': 3,
 'CIV Cases (+)': 133,
 'CIV Deaths (+)': 2}

In [21]:
#hide
dft_ct_cases = dft_cases.groupby(COL_REGION)[dt_cols].sum()
dft_ct_new_cases = dft_ct_cases.diff(axis=1).fillna(0).astype(int)
dft_ct_new_cases.head()

1/22/20  1/23/20  1/24/20  1/25/20  1/26/20  1/27/20  1/28/20  \
Country/Region                                                                  
Afghanistan           0        0        0        0        0        0        0   
Albania               0        0        0        0        0        0        0   
Algeria               0        0        0        0        0        0        0   
Andorra               0        0        0        0        0        0        0   
Angola                0        0        0        0        0        0        0   

                1/29/20  1/30/20  1/31/20  ...  3/28/20  3/29/20  3/30/20  \
Country/Region                             ...                              
Afghanistan           0        0        0  ...        0       10       50   
Albania               0        0        0  ...       11       15       11   
Algeria               0        0        0  ...       45       57       73   
Andorra               0        0        0  ...       41       26       36   
Angola                0        0        0  ...        1        2        0   

                3/31/20  4/1/20  4/2/20  4/3/20  4/4/20  4/5/20  4/6/20  
Country/Region                                                           
Afghanistan           4      63      36       8      18      50      18  
Albania              20      16      18      27      29      28      16  
Algeria             132     131     139     185      80      69     103  
Andorra               6      14      38      11      27      35      24  
Angola                0       1       0       0       2       4       2  

[5 rows x 76 columns]

In [22]:
#hide_input
template = Template(base_url+overview)
html = template.render(
    D=summary, table=df_table,  # REMOVE .head(20) to see all values
    newcases=dft_ct_new_cases.loc[:, dt_cols[LAST_DATE_I-50]:dt_cols[LAST_DATE_I]],
    np=np, pd=pd, enumerate=enumerate)
HTML(f'<div>{html}</div>')